<a href="https://colab.research.google.com/github/crisgnecco/python-projects/blob/main/blur/BlurBoxFuncionando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#@title ## 0.1 Parámetros de ejecución
#@markdown ---
#@markdown ### Especifique la URL de la imagen:
url_imagen = "https://github.com/wvaliente/SOA_HPC/blob/main/unlam.jpg?raw=true" #@param {type:"string"}

#@markdown ---
# Leo la imagen desde internet.
#!wget https://github.com/wvaliente/SOA_HPC/blob/main/unlam.jpg?raw=true -O imagen.jpg

# TODO: Mejorar información y resultado de ejecución. 
!wget {url_imagen} -O imagen.jpg

#descargo archivo de kernel CUDA //TODO: pincha al leer el archivo
#!wget {"https://github.com/crisgnecco/python-projects/blob/db78e7a99ce639b9e1bfe9d2fb78d4fd640b2060/blur/gaussian_blur.cu"}

--2021-06-30 15:43:50--  https://github.com/wvaliente/SOA_HPC/blob/main/unlam.jpg?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/wvaliente/SOA_HPC/raw/main/unlam.jpg [following]
--2021-06-30 15:43:50--  https://github.com/wvaliente/SOA_HPC/raw/main/unlam.jpg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wvaliente/SOA_HPC/main/unlam.jpg [following]
--2021-06-30 15:43:50--  https://raw.githubusercontent.com/wvaliente/SOA_HPC/main/unlam.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147032 (144K) [image/j


---
## 1.Instala en el cuaderno el módulo CUDA de Python.

In [1]:
!pip install pycuda

     |████████████████████████████████| 1.7MB 36.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627502 sha256=700f44aab08522418c05f06c4cb9e7bf1c1ca8b71a0781114b80c73f68dfa00f
  Stored in directory: /root/.cache/pip/wheels/d5/55/64/fd4dddcc5f1c25eebd90b5291c3769101dc978c70165685512
Successfully built pycuda
  Created wheel for pytools: filename=pytools-2021.2.7-py2.py3-none-any.whl size=60644 sha256=06f5d1026ff1829d57886b0f012f42bf110fd7246d194eff0b34225f1b9bfb8d
  Stored in directory: /root/.cache/pip/wheels/a0/b5/e5/e65d25997fd77729b9aa214645add18688483e48bbcbab6ffc
Successfully built pytools


## 2. Ejecutar Blur con CUDA de Python.
### - se requiere archivo gaussian_blur.cu 
### - para ajustar la intensidad del blur, cambiar la variable "sigma"// esto se podria parametrizar lindo como hizo el profe


In [14]:
# /**
# * @author Gia Duy DUONG <giaduy.duong@student.lut.fi>
# * @project Gaussian Blur - GPGPU Computing assignment
# * @copyright Copyright (c) 2019 Gia Duy DUONG
# */

# ########################### #
# import cuda & other modules #
# ########################### #
import pycuda.autoinit
import pycuda.driver as drv
import pycuda.compiler as compiler
import numpy as np
import math
import sys
import timeit
from PIL import Image

# ############### #
# check arguments #
# ############### #
try:
    input_image = str('imagen.jpg')
    output_image = str('result.jpg')
except IndexError:
    sys.exit("No input/output image")


# ################################################# #
# load image in to array and extract color channels #
# ################################################# #
try:
    img = Image.open(input_image)
    input_array = np.array(img)
    red_channel = input_array[:, :, 0].copy()
    green_channel = input_array[:, :, 1].copy()
    blue_channel = input_array[:, :, 2].copy()
except FileNotFoundError:
    sys.exit("Cannot load image file")


# ######################################## #
# generate gaussian kernel (size of N * N) #
# ######################################## #
sigma = 5  # standard deviation of the distribution
kernel_width = int(3 * sigma)
if kernel_width % 2 == 0:
    kernel_width = kernel_width - 1  # make sure kernel width only sth 3,5,7 etc

# create empty matrix for the gaussian kernel #
kernel_matrix = np.empty((kernel_width, kernel_width), np.float32)
kernel_half_width = kernel_width // 2
for i in range(-kernel_half_width, kernel_half_width + 1):
    for j in range(-kernel_half_width, kernel_half_width + 1):
        kernel_matrix[i + kernel_half_width][j + kernel_half_width] = (
                np.exp(-(i ** 2 + j ** 2) / (2 * sigma ** 2))
                / (2 * np.pi * sigma ** 2)
        )
gaussian_kernel = kernel_matrix / kernel_matrix.sum()


# #################################################################### #
# calculate the CUDA threats/blocks/gird base on width/height of image
# #################################################################### #
height, width = input_array.shape[:2]
dim_block = 32
dim_grid_x = math.ceil(width / dim_block)
dim_grid_y = math.ceil(height / dim_block)

# load CUDA code
mod = compiler.SourceModule(open('gaussian_blur.cu').read())
apply_filter = mod.get_function('applyFilter')

# ##################
# apply the  filter
# ##################
# start time
time_started = timeit.default_timer()
for channel in (red_channel, green_channel, blue_channel):
    apply_filter(
        drv.In(channel),
        drv.Out(channel),
        np.uint32(width),
        np.uint32(height),
        drv.In(gaussian_kernel),
        np.uint32(kernel_width),
        block=(dim_block, dim_block, 1),
        grid=(dim_grid_x, dim_grid_y)
    )
# end time
time_ended = timeit.default_timer()


# ####################################################################### #
# create the output array with the same shape and type as the input array #
# ####################################################################### #
output_array = np.empty_like(input_array)
output_array[:, :, 0] = red_channel
output_array[:, :, 1] = green_channel
output_array[:, :, 2] = blue_channel

# save result image
Image.fromarray(output_array).save(output_image)

# display total time
print('Total processing time: ', time_ended - time_started, 's')


Total processing time:  0.00683113099989896 s
